# 02407 Stochastic Processes exam 2025

### Question 1

This is a model for the development of cases

\begin{equation}
X_{n+1} = \min\{6,X_n-D_n+A_n\}
\end{equation}

### Question 2

The following is the transition density matrix

In [1]:
import numpy as np
import math
from scipy.stats import binom, poisson

CAP = 6
lam_A = 2.5       
p_complete = 1/3   

states = range(CAP + 1)

def build_transition_matrix():
    P = np.zeros((CAP + 1, CAP + 1))

    for i in states:
        for d in range(i + 1):
            p_d = binom.pmf(d, i, p_complete)
            r = i - d  #remaining after completions

            
            for j in range(CAP):
                a_needed = j - r
                if a_needed >= 0:
                    P[i, j] += p_d * poisson.pmf(a_needed, lam_A)

            # j = 6 (capacity)
            P[i, CAP] += p_d * poisson.sf(CAP - r - 1, lam_A)

    return P


Now we find the probability of having 5 cases under investigation in 2030 (ie. 5 years from initial point)

In [2]:

def prob_at_least_5_start_2030(x0=2):
    P = build_transition_matrix()
    #initial distribution (start of 2025)
    pi = np.zeros(CAP + 1)
    pi[x0] = 1.0

    for _ in range(5):
        pi = pi @ P

    return pi[5] + pi[6], pi, P

if __name__ == "__main__":
    prob, pi_2030, P = prob_at_least_5_start_2030()

    np.set_printoptions(precision=6, suppress=True)
    print("P(X_2030 >= 5 | X_2025 = 2) =", prob)
    print("Distribution at start of 2030:", pi_2030)


P(X_2030 >= 5 | X_2025 = 2) = 0.7400685690396935
Distribution at start of 2030: [0.001317 0.00935  0.033331 0.078881 0.137052 0.182394 0.557674]


### Question 3

Probality of hitting the absorbing state in 10 steps (ie. having one year with zero cases)

In [3]:
P_trans = build_transition_matrix()
def prob_hit_zero_within_T(P, x0=2, T=10):

    #make 0 absorbing
    P_abs = P.copy()
    P_abs[0, :] = 0.0
    P_abs[0, 0] = 1.0

    #initial distribution concentrated at x0
    pi0 = np.zeros(CAP + 1)
    pi0[x0] = 1.0

    #after T steps the probability mass in state 0 equals P(hit 0 by time T)
    
    piT = pi0 @ np.linalg.matrix_power(P_abs, T)
    return piT[0]

print(prob_hit_zero_within_T(P_trans,x0=2, T=10))


0.022642632237199565


### Question 4

In [5]:
import numpy as np

CAP_4 = 7
lam_A = 2.5       
p_complete = 1/3   

states_q4 = range(CAP_4 + 1)

def build_transition_matrix_q4():
    P = np.zeros((CAP_4 + 1, CAP_4 + 1))

    for i in states_q4:
        for d in range(i + 1):
            p_d = binom.pmf(d, i, p_complete)
            r = i - d  #remaining after completions

            
            for j in range(CAP_4):
                a_needed = j - r
                if a_needed >= 0:
                    P[i, j] += p_d * poisson.pmf(a_needed, lam_A)

            # j = 6 (capacity)
            P[i, CAP_4] += p_d * poisson.sf(CAP_4 - r - 1, lam_A)

    return P

P_trans_q4 = build_transition_matrix_q4()


In [6]:
def mean_and_var_to_hit_state(P, target=7):

    P = np.asarray(P, dtype=float)
    n = P.shape[0]

    #non-target states
    S = [i for i in range(n) if i != target]

    Q = P[np.ix_(S, S)]               # transitions among non-target states

    I = np.eye(len(S))

    #solve for mean: (I - Q) m_S = 1, where Q is P restricted to transitions among non-target states
    A = I - Q
    ones = np.ones(len(S))
    m_S = np.linalg.solve(A, ones)

    #solve for second moment: (I - Q) s_S = 1 + 2 Q m_S
    rhs_s = ones + 2.0 * (Q @ m_S)
    s_S = np.linalg.solve(A, rhs_s)

    #full vectors
    m = np.zeros(n)
    s = np.zeros(n)
    m[target] = 0.0
    s[target] = 0.0
    m[S] = m_S
    s[S] = s_S

    var = s - m**2
    return m, var

m, var = mean_and_var_to_hit_state(P_trans_q4, target=7)
print("Mean from state 2: ", m[2])
print("Var from state 2:", var[2])


Mean from state 2:  4.584017356764811
Var from state 2: 9.304507521627542


### Question 5

In [38]:

def prob_hit_a_before_b(P, a=0, b=6, start=2):

    P = np.asarray(P, dtype=float)
    n = P.shape[0]

    #boundary conditions:
    if start == a:
        return 1.0
    if start == b:
        return 0.0

    S = [i for i in range(n) if i not in (a, b)]  #unknown states
    idx = {s:i for i, s in enumerate(S)}

    Q = P[np.ix_(S, S)]

    r = P[S, a]

    A = np.eye(len(S)) - Q

    uS = np.linalg.solve(A, r)
    
    return float(uS[idx[start]])

# Example:
u2 = prob_hit_a_before_b(P_trans_q4, a=0, b=7, start=2)
u2


0.01909937895612454

### Question 6

In [19]:


def stationary_distribution_linear(P, tol=1e-12):

    P = np.asarray(P, dtype=float)
    n = P.shape[0]
    #solve (P^T - I) pi = 0 with constraint sum(pi)=1
    A = P.T - np.eye(n)
    A[-1, :] = 1.0     
    b = np.zeros(n)
    b[-1] = 1.0

    pi = np.linalg.solve(A, b)
    pi[np.abs(pi) < tol] = 0.0

    pi = np.clip(pi, 0.0, None)
    s = pi.sum()

    pi /= s
    return pi


pi1 = stationary_distribution_linear(P)
print("Stationary distribution (linear solve):", pi1)
xs = np.arange(7)
mean = (pi1 * xs).sum()
varx = (pi1 * xs**2).sum() - mean**2

print("Long run mean:", mean)
print("Long run variance:", varx)


Stationary distribution (linear solve): [0.001069 0.008032 0.030106 0.074286 0.133326 0.181687 0.571494]
Long run mean: 5.161806196196238
Long run variance: 1.4019661138322164


In [17]:
pi1 = stationary_distribution_linear(P)
C_ev = np.sum(pi1*np.arange(pi1.shape[0]))
C_ev_sq = np.sum(pi1*np.arange(pi1.shape[0])**2)
C_ev_sq-C_ev**2

1.40196611383222

## Q7

In [32]:
def build_transition_matrix_q7():
    P_q4 = build_transition_matrix_q4()
    P_q4[7,:] = 0.0
    P_q4[7,7] = 1.0
    return P_q4

In [33]:
P_trans_q7 = build_transition_matrix_q7()

In [36]:
P_10 = np.linalg.matrix_power(P_trans_q7,10)
P_10[2,7]

0.9491380230185796

In [45]:
CAP_4 = 7
accept_rate = 0.022
incident_rate = 25
lam_A = incident_rate*accept_rate   
p_complete = 1/3   

states_q4 = range(CAP_4 + 1)

def build_transition_matrix_q4():
    P = np.zeros((CAP_4 + 1, CAP_4 + 1))

    for i in states_q4:
        for d in range(i + 1):
            p_d = binom.pmf(d, i, p_complete)
            r = i - d  #remaining after completions

            
            for j in range(CAP_4):
                a_needed = j - r
                if a_needed >= 0:
                    P[i, j] += p_d * poisson.pmf(a_needed, lam_A)

            # j = 6 (capacity)
            P[i, CAP_4] += p_d * poisson.sf(CAP_4 - r - 1, lam_A)

    return P
def build_transition_matrix_q7():
    P_q4 = build_transition_matrix_q4()
    P_q4[7,:] = 0.0
    P_q4[7,7] = 1.0
    return P_q4
P_trans_q7 = build_transition_matrix_q7()
P_10 = np.linalg.matrix_power(P_trans_q7,10)
(P_10[2,7],0.99)

(0.010430484091193622, 0.99)

In [ ]:
import numpy as np
from scipy.stats import binom, poisson

CAP_4 = 7
incident_rate = 25
p_complete = 1/3

def build_transition_matrix_q4(CAP, lam_A, p_complete):
    P = np.zeros((CAP + 1, CAP + 1))
    states = range(CAP + 1)

    for i in states:
        # d = number completed among i (Binomial)
        for d in range(i + 1):
            p_d = binom.pmf(d, i, p_complete)
            r = i - d  # remaining after completions

            # j = 0..CAP-1: exact arrivals needed
            for j in range(CAP):
                a_needed = j - r
                if a_needed >= 0:
                    P[i, j] += p_d * poisson.pmf(a_needed, lam_A)

            # j = CAP: overflow (>= CAP - r arrivals)
            P[i, CAP] += p_d * poisson.sf(CAP - r - 1, lam_A)

    return P

def build_transition_matrix_q7(CAP, lam_A, p_complete):
    P = build_transition_matrix_q4(CAP, lam_A, p_complete)
    P[CAP, :] = 0.0
    P[CAP, CAP] = 1.0  # make CAP absorbing
    return P

def p_reach_cap_in_t(accept_rate, t=10, start=2, CAP=CAP_4, incident_rate=incident_rate, p_complete=p_complete):
    lam_A = incident_rate * accept_rate
    P = build_transition_matrix_q7(CAP, lam_A, p_complete)
    P_t = np.linalg.matrix_power(P, t)
    return float(P_t[start, CAP])

def find_accept_rate_for_target(target=0.99, t=10, start=2, lo=0.0, hi=1.0, tol=1e-6, max_iter=80):
    f = lambda a: p_reach_cap_in_t(a, t=t, start=start) - target

    flo, fhi = f(lo), f(hi)
    if flo >= 0:
        return lo, p_reach_cap_in_t(lo, t=t, start=start)
    if fhi <= 0:
        raise ValueError(f"Target not reachable on [{lo},{hi}]. "
                         f"At hi={hi}, prob={p_reach_cap_in_t(hi, t=t, start=start):.6f}")

    # Bisection (assumes prob increases with accept_rate)
    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        fmid = f(mid)
        if abs(fmid) <= tol:
            return mid, p_reach_cap_in_t(mid, t=t, start=start)
        if fmid < 0:
            lo = mid
        else:
            hi = mid

    mid = 0.5 * (lo + hi)
    return mid, p_reach_cap_in_t(mid, t=t, start=start)

# Example:
a_star, prob_star = find_accept_rate_for_target(target=0.01, t=10, start=2)
print("accept_rate* =", a_star)
print("P_10[2,7]    =", prob_star)